
Email

This is an update on Dec C-NOS Actual Report. Please feel free to contact me if you have any questions.
 - As Japan MO total, Dec C-NOS devliered OO IYA with OOO $MM.
 - With this, 1H to Date C-NOS is OOO IYA with OOOO $MM which is -Opt vs. 1H target (OOO IYA).
     - [By Category] Over-delivery in Baby (+4% vs. 1H Target), Hair (+1%), under-delivery in Laundry (-9%), FE (-10%),   
                       Dish (-6%), Air (-14%),  Fem (-16%), Shave (-10%), Oral (-31%), Braun (-1%), POC (-40%).
     - [By Team] Under-delivery in Div1 (-8%), Div2 (-6%), D-com (-12%), Electro (-8%).
     - [By Top Customer] Over-delivery in Rakuten (+2%), TSURUHA (0%), under-delivery in WELCIA (-10%), CAINZ (-12%), MCC 
                          (-4%), SUGI (-6%), AEON (-5%), COSMOS (-21%), COSTCO (-18%), DONKI (-18%). 


Question
1. 1st Table 
  (SQL) Create SQL to show By Category 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By Category] part
2. 2nd Table
  (SQL) Create SQL to show By Team 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By team] part
3. 3rd Table
  (SQL) Create SQL to show By Top Customer 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By Top Customer] part
 

In [0]:
%sql
-- カテゴリーごとに C-NOS IYA と $MM を算出するクエリ

SELECT
  category,                                          -- カテゴリー（Air, Hair など）

  -- 今年のC-NOS（整数で集計）
  SUM(CAST(forecast_act_est AS INT)) AS cnos,

  -- 昨年のC-NOS（整数で集計）
  SUM(CAST(ya_actual AS INT)) AS cnos_ya,

  -- C-NOS IYA = 今年 ÷ 昨年 × 100（小数点1桁）
  ROUND(
    CASE 
      WHEN SUM(CAST(ya_actual AS INT)) = 0 THEN NULL   -- 昨年ゼロの場合は割り算回避
      ELSE 100.0 * 
           SUM(CAST(forecast_act_est AS INT)) 
           / SUM(CAST(ya_actual AS INT))
    END,
    1                                                 -- 小数点第1位
  ) AS cnos_iya,

  -- C-NOS $MM（今年）：値 / 150.1 / 1000 / 1000（小数点1桁）
  ROUND(
    SUM(CAST(forecast_act_est AS INT)) / 150.1 / 1000.0 / 1000.0,
    1
  ) AS cnos_mm

FROM hive_metastore.japan_gold.cnos_vw
WHERE
  account = 'C-NOS'                                   -- C-NOSのみ対象
  -- ▼分析対象期間をここで指定してください ▼
  -- 例1：FY2526 上期
   AND period = 'FY2526 1H'
  --
  -- 例2：2025年7月〜12月
   AND month BETWEEN '2025M07' AND '2025M11'
  --
  -- 例3：特定四半期（JAS, OND, JFM, AMJ）
  -- AND quarter = 'JAS'
  -- ▲利用時にどれか一つを有効化して期間を絞ってください ▲

GROUP BY
  category
ORDER BY
  category;